In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
import re
import tensorflow_hub as hub
from tokenization import tokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsevr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsevr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('IMDB dataset.csv')
# df['content'] = df.content.astype(str)
# df['sentiment'] = df.sentiment.astype(str)
# df['sentiment'] = df.sentiment.apply(lambda x: 'negative' if x == 'empty' or x == 'sadness' or x == 'worry' or x == 'hate' or x == 'boredom' or x == 'anger' \
#                         else 'neutral' if x == 'neutral' \
#                             else 'positive')
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df['review'] = df.review.apply(lambda x: re.sub(r'@[^ ]*',r'',x))


def preprocess(text):
    t = text.lower()
    # t = re.sub('_',r'',t)
    # t = re.sub('\d+',r'',t)
    t = re.sub(r'@[^ ]*',r'',t)
    t = re.sub(r'\W+',r' ',t)
    t = re.sub(r'(https|quot|http)', '', t)
    t = re.sub(r'\b(?!(?:ai)\b)\w{3}\b','', t)
    t = re.sub(r'http?://\S+|www\.\S+','',t)
    stopwords_list = stopwords.words('english')
    txt = ' '.join([word for word in t.split() if word not in stopwords_list])
    return txt

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(email))])
                 .strip() for email in df['review']]


texts = df.prepro.values
labels = df.sentiment.values


In [4]:
ohe = OneHotEncoder(categories=[['positive','negative']])
labels = ohe.fit_transform(df.sentiment.to_numpy().reshape(-1,1)).toarray()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels,test_size=0.25)

In [6]:
from gensim.models import KeyedVectors

# Load GloVe embeddings (50-dimensional vectors in this example)
glove_path = 'glove.6B.300d.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, encoding='utf8',no_header=True)


In [7]:
# from keras.preprocessing.text import Tokenizer

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)

from keras.preprocessing.sequence import pad_sequences

max_sequence_length = 100  # Set your desired sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')


from keras.layers import Embedding

embedding_dim = 300  # Use the same dimension as your pretrained embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

embedding_layer = Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=100,
    trainable=False  # Set to True if you want to fine-tune embeddings
)



In [8]:
model = keras.Sequential()

model.add(embedding_layer)
model.add(layers.Dropout(0.4))
# model.add(layers.Conv1D(32,3, padding='same', activation='relu'))
# model.add(layers.MaxPool1D())
model.add(layers.Reshape((-1,300)))
model.add(layers.Bidirectional(layers.GRU(128,activation='tanh')))
# layers.Dropout(0.4),
# layers.Dense(64,activation='relu',kernel_initializer="he_normal"),
model.add(layers.Dropout(0.4))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics='accuracy')


In [9]:
test = tokenizer.texts_to_sequences(X_test)
test_sequences = pad_sequences(test, maxlen=100, padding='post')

In [10]:
history = model.fit(padded_sequences, y_train, epochs=20, validation_data=(test_sequences, y_test))

Epoch 1/20
1172/1172 [==============================] - 104s 86ms/step - loss: 0.5669 - accuracy: 0.6930 - val_loss: 0.4316 - val_accuracy: 0.8139
Epoch 2/20
1172/1172 [==============================] - 128s 109ms/step - loss: 0.4192 - accuracy: 0.8130 - val_loss: 0.4139 - val_accuracy: 0.8306
Epoch 3/20
1172/1172 [==============================] - 143s 122ms/step - loss: 0.3885 - accuracy: 0.8304 - val_loss: 0.4335 - val_accuracy: 0.8363
Epoch 4/20
1172/1172 [==============================] - 147s 125ms/step - loss: 0.3697 - accuracy: 0.8377 - val_loss: 0.4019 - val_accuracy: 0.8462
Epoch 5/20
1172/1172 [==============================] - 148s 126ms/step - loss: 0.3580 - accuracy: 0.8420 - val_loss: 0.4571 - val_accuracy: 0.8377
Epoch 6/20
1172/1172 [==============================] - 147s 126ms/step - loss: 0.3423 - accuracy: 0.8520 - val_loss: 0.3405 - val_accuracy: 0.8602
Epoch 7/20
1172/1172 [==============================] - 144s 123ms/step - loss: 0.3296 - accuracy: 0.8561 - val_l

In [13]:
model.save('new.h5')

In [14]:
test = ['i feel excited about my doctor appointment today']
classes = ['positive', 'negative']

def predict(text):
    test_seq = tokenizer.texts_to_sequences(text)
    pad_test = pad_sequences(test_seq, maxlen=max_sequence_length, padding='post')
    model_prediction = model.predict(pad_test)
    prediction = classes[np.argmax(model_prediction)]
    return prediction

# prediction = model.predict(pad_test)
# df.sentiment.unique()[np.argmax(prediction)]